<a href="https://colab.research.google.com/github/taqihaider7/C4AI-BIRDS-CUDA-Programming/blob/master/BIRDS_MINI_COHORT_3_CUDA_Essentials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc_plugin

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpak7sbk30".


In [ ]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 1, thread: 0
Hello from block: 1, thread: 1



## Mandatory CUDA Hello World

In [ ]:
! touch hello_world.cu
"""
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

__global__ void helloFromGPU()
{
    /*
        We can't use cout in device. Why?
        Because The GPU does not have access to standard
        output streams like std::cout, which are managed
        by the host(CPU) operating system.
    */
    printf("Hello World from GPU!\n");
}

int main()
{
    // Print from host
    cout<<"Hello World from CPU!"<<"\n";

    /*
        Launch a kernel on the GPU with one thread to
        print from GPU
    */
    helloFromGPU<<<1, 1>>>();

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    return 0;
}
"""

'\n#include <iostream>\n#include <cuda_runtime.h>\n\nusing namespace std;\n\n__global__ void helloFromGPU()\n{\n    /*\n        We can\'t use cout in device. Why?\n        Because The GPU does not have access to standard\n        output streams like std::cout, which are managed\n        by the host(CPU) operating system.\n    */\n    printf("Hello World from GPU!\n");\n}\n\nint main()\n{\n    // Print from host\n    cout<<"Hello World from CPU!"<<"\n";\n\n    /*\n        Launch a kernel on the GPU with one thread to\n        print from GPU\n    */\n    helloFromGPU<<<1, 1>>>();\n\n    // Wait for GPU to finish before accessing on host\n    cudaDeviceSynchronize();\n\n    return 0;\n}\n'

In [ ]:
! nvcc hello_world.cu -o hello_world
! ./hello_world

/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x1b): undefined reference to `main'
collect2: error: ld returned 1 exit status
/bin/bash: line 1: ./hello_world: No such file or directory


In [ ]:
! touch addition.cu
"""
#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__ void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<3, 256>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}
"""

'\n#include <iostream>\n#include <math.h>\n\n// Kernel function to add the elements of two arrays\n__global__ void add(int n, float *x, float *y)\n{\n  int index = blockIdx.x * blockDim.x + threadIdx.x;\n  int stride = blockDim.x * gridDim.x;\n  for (int i = index; i < n; i += stride)\n    y[i] = x[i] + y[i];\n}\n\nint main(void)\n{\n  int N = 1<<20;\n  float *x, *y;\n\n  // Allocate Unified Memory – accessible from CPU or GPU\n  cudaMallocManaged(&x, N*sizeof(float));\n  cudaMallocManaged(&y, N*sizeof(float));\n\n  // initialize x and y arrays on the host\n  for (int i = 0; i < N; i++) {\n    x[i] = 1.0f;\n    y[i] = 2.0f;\n  }\n\n  // Run kernel on 1M elements on the GPU\n  add<<<3, 256>>>(N, x, y);\n\n  // Wait for GPU to finish before accessing on host\n  cudaDeviceSynchronize();\n\n  // Check for errors (all values should be 3.0f)\n  float maxError = 0.0f;\n  for (int i = 0; i < N; i++)\n    maxError = fmax(maxError, fabs(y[i]-3.0f));\n  std::cout << "Max error: " << maxError << s

In [ ]:
! nvcc addition.cu -o addition
! ./addition

/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x1b): undefined reference to `main'
collect2: error: ld returned 1 exit status
/bin/bash: line 1: ./addition: No such file or directory
